In [1]:
import pandas as pd

In [2]:
pd.set_option('display.float_format', '{:.3f}'.format)
#pd.reset_option('display.float_foramt')

In [3]:
# csv 파일 읽기
file_name = '(250618)branch_rawdata.csv'

df_input = pd.read_csv(file_name, encoding='cp949', thousands=',')

In [4]:
df_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 64 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   연번                                  48 non-null     int64  
 1   기관명                                 48 non-null     object 
 2   면적                                  48 non-null     float64
 3   과수(전체)                              48 non-null     int64  
 4   센터수                                 48 non-null     int64  
 5   과수(감독전체)                            48 non-null     int64  
 6   과수(근로전체)                            48 non-null     int64  
 7   과수(산안전체)                            44 non-null     float64
 8   전체정원                                48 non-null     int64  
 9   감독관                                 48 non-null     int64  
 10  근로                                  47 non-null     float64
 11  산안                                  44 non-null

In [33]:
df_input.columns

Index(['연번', '기관명', '면적', '과수(전체)', '센터수', '과수(감독전체)', '과수(근로전체)', '과수(산안전체)',
       '전체정원', '감독관', '근로', '산안', '관할시군구수', '인구수', '사업체수', '종사자수', '신고사건접수건수',
       '신고사건처리건수', '평균처리일', '지연처리율', '감시단속적근로자적용제외승인', '특별연장근로인가건수',
       '노사협의회및노조설립·변경·해산신고건수', '취업규칙심사건수', '퇴직연금규약접수건수',
       '15세미만취직인허18세미만야간휴일근로 인가', '임산부야간휴일근로인가', '파견사업인가', '최저임금적용제외인가',
       '사내공동근로복지기금인가', '우리사주조합설립및해산신고', '근로감독건수', '정기감독실시업체', '수시감독실시업체',
       '특별감독실시업체', '사업장감독결과금품청산법위반건수', '근로시간위반건수(제50조,제53조)', '휴가위반건수(제60조)',
       '근로조건미명시건수', '부당노동행위위반(제81조)', '직장내괴롭힘적발건수', '현장점검횟수', '현장점검의날지도건수',
       '교육·컨설팅횟수', '안전보건감독건수', '안전보건진단·개선계획수립명령건수(진단)',
       '안전보건진단·개선계획수립명령건수(개선계획)', '안전인증·안전검사·유해위험방지계획서조치건수(안정인증)',
       '안전인증·안전검사·유해위험방지계획서조치건수(안전검사)', '안전인증·안전검사·유해위험방지계획서조치건수(위해위험방지계획서)',
       '산업재해조사표입력건수', '점검·감독', '점검·감독현황(위반사업장)', '점검·감독현황(행정조치)',
       '점검·감독현황(사법조치)', '중대재해', '중대산업사고', '중대재해 작업중지', '사고재해자수', '사망만인율',
       '사고사망만인율', '부상자수', '재해자수', '재해율'],
      dtype='object')

In [34]:
# 감독관 컬럼을 타겟(y)으로 설정
y = df_input['감독관']

# 특성 데이터(x) 만들기
x = df_input[['감독관', '면적', '과수(감독전체)', '관할시군구수', '인구수', '사업체수', '종사자수', '신고사건접수건수', '신고사건처리건수', '평균처리일', '지연처리율', '근로감독건수', '정기감독실시업체',
              '수시감독실시업체', '사업장감독결과금품청산법위반건수', '현장점검횟수', '현장점검의날지도건수', '교육·컨설팅횟수', '안전보건감독건수', '산업재해조사표입력건수', '점검·감독' ,
              '중대재해', '사고재해자수', '사망만인율', '사고사망만인율', '부상자수', '재해자수', '재해율']]



In [6]:
# 면적은 2차원 값이므로, 1차원 값인 거리를 추가
x['거리'] = x['면적'] ** 0.5

C:\Users\barkl\AppData\Local\Temp\ipykernel_11732\3818797131.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['거리'] = x['면적'] ** 0.5


In [35]:
columns_to_normalize = [col for col in x.columns if col != '감독관']

# 3. 새로운 데이터프레임을 생성하여 '감독관'으로 나눈 값들을 저장
# .copy()를 사용하여 원본 x 데이터프레임이 수정되지 않도록 합니다.
x2 = x.copy()

# 4. '감독관' 컬럼의 값으로 각 행의 다른 모든 컬럼을 나눕니다.
# .div() 메서드를 사용하며, axis=0은 행 방향(즉, 각 행의 '감독관' 값으로)으로 나눔을 의미합니다.
# 주의: 만약 '감독관' 컬럼에 0이 있다면 ZeroDivisionError가 발생하거나 NaN(Not a Number)이 될 수 있습니다.
# 실제 데이터에서 이 부분을 어떻게 처리할지 고려해야 합니다 (예: 0을 다른 값으로 대체하거나 해당 행을 건너뛰기).
x2[columns_to_normalize] = x[columns_to_normalize].div(x['감독관'], axis=0)

In [36]:
x2.head()

,감독관,면적,과수(감독전체),관할시군구수,인구수,사업체수,종사자수,신고사건접수건수,신고사건처리건수,평균처리일,...,안전보건감독건수,산업재해조사표입력건수,점검·감독,중대재해,사고재해자수,사망만인율,사고사망만인율,부상자수,재해자수,재해율
0,146,0.651,0.055,0.027,6898.349,659.877,7663.582,80.747,74.404,0.261,...,3.274,13.829,3.274,0.534,18.041,0.003,0.001,17.925,20.712,0.002
1,88,0.449,0.057,0.011,6323.932,689.727,8306.420,138.932,124.125,0.587,...,3.841,15.932,3.841,0.102,19.239,0.005,0.002,19.023,21.534,0.003
2,82,1.126,0.049,0.049,21343.671,897.793,8972.293,132.146,120.902,0.470,...,5.280,28.366,5.280,0.280,34.854,0.010,0.003,34.524,46.756,0.007
3,71,1.310,0.056,0.056,18642.859,745.056,7449.310,115.958,105.817,0.705,...,4.648,17.451,4.648,0.197,27.859,0.009,0.004,27.592,32.211,0.006
4,75,1.112,0.053,0.040,18051.933,804.253,9597.893,130.720,122.040,0.469,...,4.213,22.867,4.213,0.053,28.973,0.010,0.003,28.707,32.933,0.005


In [37]:
x3= x2[['면적', '과수(감독전체)', '관할시군구수', '인구수', '사업체수', '종사자수', '신고사건접수건수', '신고사건처리건수', '평균처리일', '지연처리율', '근로감독건수', '정기감독실시업체',
        '수시감독실시업체', '사업장감독결과금품청산법위반건수', '현장점검횟수', '현장점검의날지도건수', '교육·컨설팅횟수', '안전보건감독건수', '산업재해조사표입력건수', '점검·감독' ,
        '중대재해', '사고재해자수', '사망만인율', '사고사망만인율', '부상자수', '재해자수', '재해율']]

In [38]:
x3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   면적                48 non-null     float64
 1   과수(감독전체)          48 non-null     float64
 2   관할시군구수            48 non-null     float64
 3   인구수               48 non-null     float64
 4   사업체수              48 non-null     float64
 5   종사자수              48 non-null     float64
 6   신고사건접수건수          48 non-null     float64
 7   신고사건처리건수          48 non-null     float64
 8   평균처리일             48 non-null     float64
 9   지연처리율             48 non-null     float64
 10  근로감독건수            48 non-null     float64
 11  정기감독실시업체          48 non-null     float64
 12  수시감독실시업체          48 non-null     float64
 13  사업장감독결과금품청산법위반건수  48 non-null     float64
 14  현장점검횟수            48 non-null     float64
 15  현장점검의날지도건수        48 non-null     float64
 16  교육·컨설팅횟수          48 non-null     float64
 17 

In [39]:
from sklearn.preprocessing import StandardScaler

# StandardScaler 객체 생성
stan_scaler = StandardScaler()

In [40]:
x_stan = pd.DataFrame(stan_scaler.fit_transform(x3), columns=x3.columns)

In [41]:
x_stan.head()

,면적,과수(감독전체),관할시군구수,인구수,사업체수,종사자수,신고사건접수건수,신고사건처리건수,평균처리일,지연처리율,...,안전보건감독건수,산업재해조사표입력건수,점검·감독,중대재해,사고재해자수,사망만인율,사고사망만인율,부상자수,재해자수,재해율
0,-0.761,-0.487,-0.977,-1.324,-0.152,0.491,-0.509,-0.515,-0.776,-0.705,...,-0.879,-1.115,-0.885,-0.126,-0.997,-0.251,-0.771,-0.991,-0.911,-0.402
1,-0.763,-0.401,-1.177,-1.399,-0.070,0.710,0.521,0.435,-0.477,-0.270,...,-0.800,-1.000,-0.805,-1.152,-0.936,-0.247,-0.692,-0.935,-0.886,-0.387
2,-0.755,-0.743,-0.711,0.565,0.502,0.937,0.401,0.373,-0.584,-0.554,...,-0.597,-0.319,-0.603,-0.729,-0.144,-0.241,-0.655,-0.138,-0.127,-0.337
3,-0.752,-0.421,-0.617,0.212,0.082,0.418,0.115,0.085,-0.370,-0.306,...,-0.686,-0.917,-0.692,-0.927,-0.499,-0.241,-0.633,-0.494,-0.565,-0.340
4,-0.755,-0.549,-0.821,0.135,0.245,1.150,0.376,0.395,-0.586,-0.564,...,-0.747,-0.620,-0.753,-1.268,-0.442,-0.240,-0.673,-0.437,-0.543,-0.357


In [42]:
x_stan.describe()

,면적,과수(감독전체),관할시군구수,인구수,사업체수,종사자수,신고사건접수건수,신고사건처리건수,평균처리일,지연처리율,...,안전보건감독건수,산업재해조사표입력건수,점검·감독,중대재해,사고재해자수,사망만인율,사고사망만인율,부상자수,재해자수,재해율
count,48.000,48.000,48.000,48.000,48.000,48.000,48.000,48.000,48.000,48.000,...,48.000,48.000,48.000,48.000,48.000,48.000,48.000,48.000,48.000,48.000
mean,-0.000,-0.000,0.000,0.000,-0.000,-0.000,-0.000,-0.000,0.000,-0.000,...,-0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000
std,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,...,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011,1.011
min,-0.763,-1.400,-1.177,-2.226,-0.874,-1.042,-1.143,-1.156,-0.852,-0.836,...,-0.879,-1.115,-0.885,-1.268,-0.997,-0.251,-0.771,-0.991,-0.911,-0.402
25%,-0.704,-0.549,-0.708,-0.488,-0.407,-0.485,-0.448,-0.434,-0.589,-0.570,...,-0.433,-0.479,-0.419,-0.696,-0.437,-0.241,-0.642,-0.434,-0.473,-0.340
50%,-0.407,-0.315,-0.386,-0.050,-0.133,-0.210,-0.181,-0.168,-0.453,-0.417,...,-0.215,-0.166,-0.219,-0.118,-0.116,-0.230,-0.417,-0.107,-0.166,-0.269
75%,0.239,0.351,0.237,0.285,0.172,0.232,0.308,0.317,0.244,0.253,...,0.235,0.245,0.231,0.248,0.198,-0.192,0.264,0.198,0.024,-0.025
max,3.507,4.916,2.828,4.978,6.316,5.910,6.106,6.093,3.883,3.653,...,6.110,5.715,6.116,4.648,6.310,6.118,4.570,6.308,4.377,6.073


In [43]:
model = LinearRegression()

# .fit(독립 변수, 종속 변수) 메서드를 호출하여 모델을 학습시킵니다.
model.fit(x_stan, y)


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [44]:
print("--- sklearn LinearRegression 결과 ---")

# 절편 (Intercept)
# model.intercept_ 에 절편값이 저장됩니다.
intercept = model.intercept_
print(f"✅ 절편 (b): {intercept:.4f}")

# 가중치 (Coefficients)
# model.coef_ 에 각 독립 변수의 가중치(계수)가 numpy 배열 형태로 저장됩니다.
coefficients = model.coef_
print("✅ 각 변수의 가중치 (w):")
# 가중치를 변수 이름과 매핑하여 출력합니다.
for feature_name, coef_value in zip(x_stan.columns, coefficients):
    print(f"   - {feature_name}: {coef_value:.4f}")

print("\n--- 수식 형태 ---")
# 추출된 값들을 사용하여 수식 형태를 출력
equation = f"y = {intercept:.4f}"
for i, feature_name in enumerate(x_stan.columns):
    equation += f" + {coefficients[i]:.4f} * {feature_name}"
print(equation)


--- sklearn LinearRegression 결과 ---
✅ 절편 (b): 65.2708
✅ 각 변수의 가중치 (w):
   - 면적: 3.1269
   - 과수(감독전체): -0.2755
   - 관할시군구수: -2.0650
   - 인구수: -7.0806
   - 사업체수: 9.1067
   - 종사자수: -13.7180
   - 신고사건접수건수: -113.4337
   - 신고사건처리건수: 125.7405
   - 평균처리일: -41.1861
   - 지연처리율: 37.7782
   - 근로감독건수: -1800.3918
   - 정기감독실시업체: 1507.3539
   - 수시감독실시업체: 318.3005
   - 사업장감독결과금품청산법위반건수: -3.3358
   - 현장점검횟수: -4.8437
   - 현장점검의날지도건수: 15.8028
   - 교육·컨설팅횟수: -1.1550
   - 안전보건감독건수: -252.0492
   - 산업재해조사표입력건수: -7.4256
   - 점검·감독: 230.8382
   - 중대재해: 22.4982
   - 사고재해자수: -1317.6164
   - 사망만인율: 149.8736
   - 사고사망만인율: -5.8940
   - 부상자수: 1295.9025
   - 재해자수: 28.8357
   - 재해율: -167.4380

--- 수식 형태 ---
y = 65.2708 + 3.1269 * 면적 + -0.2755 * 과수(감독전체) + -2.0650 * 관할시군구수 + -7.0806 * 인구수 + 9.1067 * 사업체수 + -13.7180 * 종사자수 + -113.4337 * 신고사건접수건수 + 125.7405 * 신고사건처리건수 + -41.1861 * 평균처리일 + 37.7782 * 지연처리율 + -1800.3918 * 근로감독건수 + 1507.3539 * 정기감독실시업체 + 318.3005 * 수시감독실시업체 + -3.3358 * 사업장감독결과금품청산법위반건수 + -4.8437 * 현장점검횟수 + 15

In [15]:
from sklearn.feature_selection import SelectKBest, f_regression

In [16]:
# X_normalized: 정규화된 특성 DataFrame
# y: 감독관(타겟) Series

# 각 특성과 타겟(y) 간의 F-값을 계산하여 통계적으로 가장 중요한 K개의 특성을 선택
# f_regression은 회귀 문제에서 사용되는 통계 테스트입니다.
selector = SelectKBest(score_func=f_regression, k=3)
x_select = selector.fit_transform(x_stan, y)

# 선택된 특성들의 이름 확인
selected_features_indices = selector.get_support(indices=True)
selected_features_names = x_stan.columns[selected_features_indices]

print("Selected features (Filter method):", selected_features_names)

# X_selected_filter는 numpy 배열이므로 필요시 다시 DataFrame으로 변환
x_select_df = pd.DataFrame(x_select, columns=selected_features_names)

Selected features (Filter method): Index(['과수(감독전체)', '근로감독건수', '정기감독실시업체'], dtype='object')


In [17]:
feature_scores = pd.DataFrame({
    'Feature': x_stan.columns, # 모든 특성 이름
    'F_Score': selector.scores_, # 각 특성의 F-값
    'P_value': selector.pvalues_
})

In [18]:
feature_scores

,Feature,F_Score,P_value
0,면적,1.649,0.206
1,과수(감독전체),415.450,0.000
2,관할시군구수,9.806,0.003
3,인구수,41.917,0.000
4,사업체수,282.511,0.000
5,종사자수,202.359,0.000
6,신고사건접수건수,215.367,0.000
7,신고사건처리건수,211.014,0.000
8,평균처리일,0.916,0.344
9,지연처리율,1.533,0.222


In [19]:

print(feature_scores.sort_values(by='F_Score', ascending=False))

             Feature  F_Score  P_value
10            근로감독건수  438.008    0.000
11          정기감독실시업체  418.447    0.000
1           과수(감독전체)  415.450    0.000
4               사업체수  282.511    0.000
12          수시감독실시업체  223.580    0.000
6           신고사건접수건수  215.367    0.000
7           신고사건처리건수  211.014    0.000
5               종사자수  202.359    0.000
24              부상자수  146.518    0.000
21            사고재해자수  145.199    0.000
25              재해자수  116.291    0.000
18       산업재해조사표입력건수   92.546    0.000
14            현장점검횟수   89.336    0.000
19             점검·감독   84.357    0.000
17          안전보건감독건수   80.599    0.000
20              중대재해   69.633    0.000
13  사업장감독결과금품청산법위반건수   65.126    0.000
15        현장점검의날지도건수   46.226    0.000
3                인구수   41.917    0.000
23           사고사망만인율   15.479    0.000
16          교육·컨설팅횟수   11.387    0.002
2             관할시군구수    9.806    0.003
26               재해율    6.023    0.018
22             사망만인율    5.073    0.029
27                거리    2

In [20]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# X_normalized: 정규화된 특성 DataFrame
# y: 감독관(타겟) Series

# Linear Regression 모델을 기반으로 RFE 수행
estimator = LinearRegression() # 중요도를 판단할 모델 (여기서는 선형 회귀)

In [21]:
# RFE 객체 생성: estimator를 사용하여 15개의 특성만 남기도록 설정
selector_rfe = RFE(estimator=estimator, n_features_to_select=15, step=1) # step=1은 한 번에 1개씩 제거

# 특성 선택 과정 실행
X_selected_wrapper = selector_rfe.fit_transform(x_stan, y)

# 선택된 특성들의 이름 확인
selected_features_rfe_indices = selector_rfe.get_support(indices=True)
selected_feature_rfe_names = x_stan.columns[selected_features_rfe_indices]

print("Selected features (Wrapper method - RFE):", selected_feature_rfe_names)

# X_selected_wrapper는 numpy 배열이므로 필요시 다시 DataFrame으로 변환
X_selected_wrapper_df = pd.DataFrame(X_selected_wrapper, columns=selected_feature_rfe_names)

Selected features (Wrapper method - RFE): Index(['면적', '인구수', '사업체수', '신고사건접수건수', '신고사건처리건수', '근로감독건수', '정기감독실시업체',
       '수시감독실시업체', '현장점검의날지도건수', '안전보건감독건수', '점검·감독', '중대재해', '사고재해자수', '부상자수',
       '거리'],
      dtype='object')


In [22]:
selector_rfe.estimator_.coef_
#selector_rfe.estimator_.intercept_

array([  -22.4384267 ,    14.8633619 ,   -15.53255209,   -15.84354033,
          45.48840797,   722.78374117,  -556.57167279,  -138.45455144,
          10.42461916,   -72.86240772,    68.89159107,     9.76843264,
       -1087.9498098 ,  1057.80849169,    33.20616887])

In [23]:
selector_rfe

,estimator,LinearRegression()
,n_features_to_select,15
,step,1
,verbose,0
,importance_getter,'auto'
,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [24]:
selector_rfe._parameter_constraints

{'estimator': [<sklearn.utils._param_validation.HasMethods at 0x1dba180cad0>],
 'n_features_to_select': [None,
 'step': [<sklearn.utils._param_validation.Interval at 0x1dba180cbf0>,
 'verbose': ['verbose'],
 'importance_getter': [str, <function callable(obj, /)>]}

In [25]:
intercept = selector_rfe.estimator_.intercept_
features_weights = selector_rfe.estimator_.coef_

# Create a DataFrame to pair feature names with their weights
weights_df = pd.DataFrame({
    'Feature': selected_features_names,
    'Weight': features_weights
})

weights_df


ValueError: All arrays must be of the same length